In [2]:
import os
import sys
from pathlib import Path

file = Path(os.path.abspath('')).resolve()
parent, root = file.parent, file.parents[1]
sys.path.append(str(root))

import import_ipynb
import json
from collections import defaultdict
import syntaxCheck as sync
import semanticCheck as smtc

In [1]:
class validate1:
    
    def __init__(self, tar_file):
        self.tar_file = tar_file
        self.extension = os.path.splitext(self.tar_file)[1]
        
        #define a dictionary storing the checking functions in value
        dictionary = {
            "File existence": sync.existence_check,
            "File size": sync.file_size_check, 
            "Metadata file existence":sync.metadata_check,
            "Json check": sync.string_value_check,
            "Json-ld check": sync.check_context,
            "Self descriptor check": smtc.self_descriptor_check, 
            "Direct property check": smtc.direct_property_check
        }
        
        #initialize the check result and a list to store the checking function
        self.functions = []
        self.result = defaultdict(list)
        
        with open ("check_list.txt") as file: 
            check_list = list(map(lambda s: s.strip(), file.readlines()))
        
        ### store the functions in a dictionary
        for x in check_list:
            if x in dictionary:
                self.functions.append(dictionary[x])
        
        for i in range(len(self.functions)):
            self.result[list(dictionary.keys())[i].rstrip("\n")].append(None)

        
    def update_output(self, term, check_result):
        self.result[term][0] = check_result[1] if len(check_result) < 3 else check_result[2]
        if len(check_result) >=3: 
            self.result[term].append(check_result[1])
    
    def validator(self):
        for methods in self.functions: 
            check_result = methods(self.tar_file, self.extension)
            if (check_result[0] == "File existence" or check_result[0] == "File_size" or check_result[0] == "Metadata file existence") and isinstance(check_result[1], bool) == False:
                self.update_output(check_result[0], check_result)
                break
#             self.update_output(check_result[0], check_result)
        output = json.dumps(self.result, indent = 4, separators=(',', ': '), sort_keys = False)
            
        print(output)

        